# Textmining
### Chunking-Grammatik zum extrahieren von Nominalphrasen (NPs)

<img src="https://avatars.githubusercontent.com/u/49121218?v=4" alt="Avatar" style="float: left; padding-right:1rem; padding-bottom: .5rem;" width=80/>

📝 [Oguzhan-Burak Bozkurt](https://github.com/0xBuro)<br/>
🗓️ Mai 2023 <br/>

Tools: <br/>
<div>

[<img src="https://simpleicons.org/icons/python.svg" alt="Python" width="28"/>](https://www.python.org/)
[<img src="https://simpleicons.org/icons/jupyter.svg" alt="Jupyter" width="28"/>](https://jupyter.org/)
<br/>

[HanTa - The Hannover Tagger](https://github.com/wartaal/HanTa)

Der Part-Of-Speech Tagger von <code>nltk</code> unterstützt derzeit nur die englische und russische Sprache.
Mit dem <code>HanTa</code> package von Prof. Dr. Christian Wartena kann Lemmatisierung und POS-tagging auch für deutsche Texte eingesetzt werden.
Das Modul führt morphologische Analysen nach dem Hidden Markov Model durch.
</div>

## To-Do:
1. [x] Suchen eines deutschen Textes von 250 - 400 Wörtern.
 - Ich verwende hier die Einführung aus der deutschen Übersetzung des Bitcoin Whitepaper PDFs von Satoshi Nakamoto mit 265 Wörtern. (https://bitcoin.org/files/bitcoin-paper/bitcoin_de.pdf)

2. [x] händische Extrahierung von Nominalphrasen -> Markierung & Anschließend Aufnahme in Baum-Tupel.
3. [x] Chunking-Grammatik mit NLTK & RegExp -> parsing.
4. [x] Extrahieren aller NPs aus dem Text.
5. [x] Precision & Recall berechnen -> Vergleich händischer Extrahierung mit Chunking-Grammatik.
6. [x] SpaCy zum extrahieren aller NPs -> Vergleich mit NLTK.
7. [x] Precision & Recall für SpaCy vs. NLTK.
8. [x] Unterschied der Ergebnisse -> Analyse/Fazit/Anmerkungen

## Nominalphrasen

### Text: Ausschnitt/Einleitung aus Satoshi Nakamotos Bitcoin Whitepaper

"Der Internethandel ist inzwischen fast vollständig von Finanzinstituten abhängig, die als vertrauenswürdige Dritte elektronische Zahlungen abwickeln. Während dieses System für die meisten Transaktionen angemessen funktioniert, leidet es dennoch unter den inhärenten Schwächen des vertrauensbasierten Modells. Nicht rückgängig zu machende Transaktionen sind nicht wirklich realisierbar, da Finanzinstitute dazu gezwungen sind, bei Streitigkeiten zu vermitteln. Die Kosten der Konfliktlösung erhöht die Transaktionskosten, was die kleinste praktikable Transaktionsgröße und damit die Möglichkeit kleiner, gelegentlicher Transaktionen einschränkt. Dazu kommen die allgemeineren Kosten des Verlustes der Fähigkeit, irreversible Zahlungen für irreversible Dienstleistungen tätigen zu können.
Mit der Möglichkeit einer Rückgängigmachung breitet sich die Notwendigkeit des Vertrauens aus. Händler müssen ihren Kunden gegenüber argwöhnisch sein
und ihnen mehr Informationen abverlangen, als sie ansonsten benötigten. Ein gewisser
Anteil an Betrügerei wird als unausweichlich hingenommen. Diese Kosten und Zahlungsunsicherheiten können bei persönlichen Interaktionen durch die Verwendung von Bargeld vermieden werden, doch es existiert kein Mechanismus, der Zahlungen ohne vertrauenswürdige Dritte über Kommunikationskanäle ermöglichte.
Was notwendig ist, ist ein Zahlungssystem, das auf kryptographischen Nachweisen
statt Vertrauen beruht und es zwei so geneigten Parteien ermöglicht, direkt und ohne
Rückgriff auf eine vertrauenswürdige dritte Partei Transaktionen untereinander abzuwi-
ckeln. Transaktionen, die praktisch nicht rückgängig gemacht werden können, schützten
Verkäufer vor Betrug, während übliche Treuhanddienste leicht implementiert werden
könnten, um Käufer zu schützen. In dieser Arbeit schlagen wir eine Lösung des Doppel-
ausgabenproblems vor, bei welcher ein Peer-to-Peer-verteilter Zeitstempelserver 
benutzt wird, um einen rechnergestützten Nachweis der chronologischen Reihenfolge von
Transaktionen zu erzeugen. Das System ist sicher, solange ehrliche Nodes kollektiv mehr CPU-Leistung kontrollieren als jede kooperierende Gruppe angreifender Nodes."

In [1]:
# Einer Liste von Strings zugeordnet, mit Angaben zu den grammatischen Bausteinen.
# Diese händische Liste können wir später mit der Liste aus der Chunking Grammatik vergleichen.

nominalphrasen_haendisch = [
    "Der Internethandel",    
    "Finanzinstituten",    
    "vertrauenswürdige Dritte",   
    "System",    
    "die meisten Transaktionen",    
    "den inhärenten Schwächen",    
    "des vertrauensbasierten Modells",           
    "Streitigkeiten",   
    "Die Kosten",       
    "die Transaktionskosten",    
    "die kleinste praktikable Transaktionsgröße",    
    "die Möglichkeit",   
    "gelegentlicher Transaktionen",    
    "die allgemeineren Kosten",    
    "des Verlustes",  
    "der Fähigkeit",  
    "irreversible Zahlungen",   
    "irreversible Dienstleistungen",   
    "der Möglichkeit",    
    "einer Rückgängigmachung",    
    "die Notwendigkeit",    
    "des Vertrauens",    
    "Händler",    
    "ihren Kunden",    
    "Informationen",    
    "Ein gewisser Anteil an Betrügerei",       
    "Kosten",    
    "Zahlungsunsicherheiten",   
    "persönlichen Interaktionen",    
    "die Verwendung",    
    "Bargeld",    
    "Mechanismus",    
    "der Zahlungen",    
    "Kommunikationskanäle",    
    "ein Zahlungssystem",    
    "eine vertrauenswürdige dritte Partei",    
    "Transaktionen",      
    "eine Lösung",    
    "des Doppelausgabenproblems",   
    "ein Peer-to-Peer-verteilter Zeitstempelserver",    
    "einen rechnergestützten Nachweis",    
    "der chronologischen Reihenfolge",      
    "Das System",    
    "Nodes",    
    "CPU-Leistung",   
    "kooperierende Gruppe",    
]

## Chunking-Grammatik regelbasiert & ML 

Für die chunking-Grammatik mit NLTK importieren wir zunächst die <code>nltk</code> Bibliothek und einige essentielle Module wie tag, tokenize und chunk.
Da das pos_tag Modul nur englisch und russisch unterstüzt, nutzen wir noch den HanoverTagger aus HanTa.

In [2]:
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.chunk import RegexpParser

from HanTa import HanoverTagger as ht

# tagger mit morphologischem Model für die deutsche Sprache
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

##### 1. Text einer Variable zuordnern

In [3]:
# Einleitung aus dem Bitcoin Whitepaper auf deutsch.

satoshis_paper = """
Der Internethandel ist inzwischen fast vollständig von Finanzinstituten abhängig, die als
vertrauenswürdige Dritte elektronische Zahlungen abwickeln. Während dieses System
für die meisten Transaktionen angemessen funktioniert, leidet es dennoch unter den
inhärenten Schwächen des vertrauensbasierten Modells. Nicht rückgängig zu machende
Transaktionen sind nicht wirklich realisierbar, da Finanzinstitute dazu gezwungen sind,
bei Streitigkeiten zu vermitteln. Die Kosten der Konfliktlösung erhöht die Transaktionskosten,
was die kleinste praktikable Transaktionsgröße und damit die Möglichkeit kleiner, 
gelegentlicher Transaktionen einschränkt. Dazu kommen die allgemeineren Kosten des Verlustes 
der Fähigkeit, irreversible Zahlungen für irreversible Dienstleistungen tätigen zu können. 
Mit der Möglichkeit einer Rückgängigmachung breitet sich die Notwendigkeit des Vertrauens aus. 
Händler müssen ihren Kunden gegenüber argwöhnisch sein und ihnen mehr Informationen abverlangen, 
als sie ansonsten benötigten. Ein gewisser Anteil an Betrügerei wird als unausweichlich hingenommen. 
Diese Kosten und Zahlungsunsicherheiten können bei persönlichen Interaktionen durch die Verwendung von 
Bargeld vermieden werden, doch es existiert kein Mechanismus, der Zahlungen ohne vertrauenswürdige 
Dritte über Kommunikationskanäle ermöglichte. Was notwendig ist, ist ein Zahlungssystem, das auf 
kryptographischen Nachweisen statt Vertrauen beruht und es zwei so geneigten Parteien ermöglicht, 
direkt und ohne Rückgriff auf eine vertrauenswürdige dritte Partei Transaktionen untereinander abzuwickeln. 
Transaktionen, die praktisch nicht rückgängig gemacht werden können, schützten
Verkäufer vor Betrug, während übliche Treuhanddienste leicht implementiert werden
könnten, um Käufer zu schützen. In dieser Arbeit schlagen wir eine Lösung des Doppelausgabenproblems vor, 
bei welcher ein Peer-to-Peer-verteilter Zeitstempelserver benutzt wird, um einen rechnergestützten Nachweis 
der chronologischen Reihenfolge von Transaktionen zu erzeugen. Das System ist sicher, 
solange ehrliche Nodes kollektiv mehr CPU-Leistung kontrollieren als jede kooperierende Gruppe angreifender Nodes.
"""

##### 2. Chunking-Grammatik bauen. Wir erstellen außerdem einen Parser unter Verwendung von regulären Ausdrücken in der Grammatik.

In [4]:
# Chunking-Grammatik für NPs
chunk = r"""
NP:    {(<ART>|<PPOSAT>)<[^N].*>*<NN>}     # Artikel, Adjektiv, Substantiv
       {<NE.*>}                            # benannte Entität
       {<PDS><NNP>}                        # Personalpronomen + Eigennamen
       {<APPR.*><NP>}                      # Präposition gefolgt von Nominalphrase
       {<PRELS|PRELAT><.*>*<NN.*>}         # relative Pronomen und Artikel
       {(<ART>|<PPOSAT>)?<ADJ\(A\)>*<NN>}  # optionale Pronomen, besitzanzeigende Pronomen, Attributiv (Adjektiv direkt vor Substantiv)
       {<NE>}                              # benannte Entität
"""

# Parser aus Chunking-Grammatik
chunk_parser = RegexpParser(chunk)

# Methode die wir später zum Extrahieren der Chunks verwenden werden
def chunk_deploy(draw, label):
    result = []
    for n in draw:
        if isinstance(n, nltk.tree.Tree):               
            if n.label() == label:
                phrase =  n.leaves()
                result.append(phrase)
    return result

##### 3. Tokenisierung, POS-Tagging, Chunking & Lemmatisierung aller Sätze. 

In [5]:
# Tokenisieren der Sätze im gesamten Text
tokenized_sents = sent_tokenize(satoshis_paper, "german")

# Tokenisieren der Wörter in den tokenisierten Sätzen
word_token = []
for word in tokenized_sents:
    word_token.append(word_tokenize(word))

# POS-Tagging
pos_tags = []
for pos in word_token:
    pos_tags.append(tagger.tag_sent(pos))

# Chunking
chunked_list = []
for element in pos_tags:
    chunked_list.append(chunk_parser.parse(element))

# Lemmetierung und Chunkdeploy aufruf mit abgeflachter Ausgabe der Verarbeitung.
result_list = []
result_tree = []
for i in chunked_list:
    result = ([(wort, pos) for (wort, lemma, pos) in i])
    result = chunk_parser.parse(result)
    for j in chunk_deploy(result,"NP"):
        result_list.append(" ".join([wort for (wort,pos) in j]))
        result_tree.append(j)

result_list

['Der Internethandel',
 'Finanzinstituten',
 'die als vertrauenswürdige Dritte elektronische Zahlungen',
 'System',
 'die meisten Transaktionen',
 'den inhärenten Schwächen',
 'des vertrauensbasierten Modells',
 'machende Transaktionen',
 'Finanzinstitute',
 'Streitigkeiten',
 'Die Kosten',
 'der Konfliktlösung',
 'die Transaktionskosten',
 'die kleinste praktikable Transaktionsgröße',
 'die Möglichkeit',
 'gelegentlicher Transaktionen',
 'die allgemeineren Kosten',
 'des Verlustes',
 'der Fähigkeit',
 'irreversible Zahlungen',
 'irreversible Dienstleistungen',
 'der Möglichkeit',
 'einer Rückgängigmachung',
 'die Notwendigkeit',
 'des Vertrauens',
 'Händler',
 'ihren Kunden',
 'Informationen',
 'Ein gewisser Anteil',
 'Betrügerei',
 'Kosten',
 'Zahlungsunsicherheiten',
 'persönlichen Interaktionen',
 'die Verwendung',
 'Bargeld',
 'Mechanismus',
 'der Zahlungen',
 'Kommunikationskanäle',
 'ein Zahlungssystem',
 'das auf kryptographischen Nachweisen statt Vertrauen beruht und es zwei s

Für eine erste Beurteilung sehen die NPs mit dieser Ausgabe gut aus. Um das Chunking-Modell und seine Qualität besser beurteilen zu könnnen, sollten wir jedoch das Modell als Tree ausgeben lassen, um auch die POS-Tags zu sehen. Wir geben einfach den result_tree mit allen Tupeln also allen Sätzen in einem aus.


In [6]:
# gibt alle geparseden Chunks zurück, Baum für Baum aus dem zuvor gesamt verschachtelten Baum
for tree in result_tree:
    print(tree)
    print(chunk_parser.parse(tree))
    print()

[('Der', 'ART'), ('Internethandel', 'NN')]
(S (NP Der/ART Internethandel/NN))

[('Finanzinstituten', 'NN')]
(S (NP Finanzinstituten/NN))

[('die', 'PRELS'), ('als', 'APPR'), ('vertrauenswürdige', 'ADJ(A)'), ('Dritte', 'NNA'), ('elektronische', 'ADJ(A)'), ('Zahlungen', 'NN')]
(S
  (NP
    die/PRELS
    als/APPR
    vertrauenswürdige/ADJ(A)
    Dritte/NNA
    elektronische/ADJ(A)
    Zahlungen/NN))

[('System', 'NN')]
(S (NP System/NN))

[('die', 'ART'), ('meisten', 'PIAT'), ('Transaktionen', 'NN')]
(S (NP die/ART meisten/PIAT Transaktionen/NN))

[('den', 'ART'), ('inhärenten', 'ADJ(A)'), ('Schwächen', 'NN')]
(S (NP den/ART inhärenten/ADJ(A) Schwächen/NN))

[('des', 'ART'), ('vertrauensbasierten', 'ADJ(A)'), ('Modells', 'NN')]
(S (NP des/ART vertrauensbasierten/ADJ(A) Modells/NN))

[('machende', 'ADJ(A)'), ('Transaktionen', 'NN')]
(S (NP machende/ADJ(A) Transaktionen/NN))

[('Finanzinstitute', 'NN')]
(S (NP Finanzinstitute/NN))

[('Streitigkeiten', 'NN')]
(S (NP Streitigkeiten/NN))

[('D

Wir können bereits eine große Übereinstimmung sehen. Unsere Chunking-Grammatik sieht gut aus und macht seine Aufgabe Nominalphrasen zu erkennen gut.

### Precision & Recall
Um das Modell etwas genauer aber relativ schnell auf Qualität zu prüfen können wir Precision & Recall berechnen.
um Precision & Recall mit True Positive, False Positive und False Negative zu berechnen bräuchten wir jedoch Testdaten. 
Wir haben zusätzlich zu unserem Chunk-deploytem Ergebnis aber unsere händisch extrahierten Daten, die für eine Berechnung ausreichend ist.
Wir rechnen die Anzahl richtig erkannter NPs, also Übereinstimmungen von nominalphrasen_haendisch und unserer result_list zusammen, wir brauchen also auch nichts weiter in Tupel oder ähnliches setzen, wobei wir so auch auf die Erkennung bzw. die POS-Tag schauen könnten.


In [7]:
# Precision berechnen
precision = 0
for np in result_list:
    if np in nominalphrasen_haendisch:
        precision += 1
precision /= len(result_list)

# Recall berechnen
recall = 0
for np in nominalphrasen_haendisch:
    if np in result_list:
        recall += 1
recall /= len(nominalphrasen_haendisch)

# Print precision and recall
print("Precision:", precision)
print("Recall:", recall)

Precision: 0.8392857142857143
Recall: 0.9565217391304348


Wir haben ziemlich hohe Werte, was in unserem Fall zum einen bedeutet, dass wir relativ viele NPs bereits händisch gefunden haben, zum anderen ist aber auch unsere Chunking-Grammatik genau so gut darin NPs zuzuordnen. In unserem Fall haben wir ein relativ solides Modell, dass NPs aus dem Satoshi Nakamoto Bitcoin Whitepaper zuordnen kann.

## SpaCy

Vergleich mit SpaCy

In [8]:
import spacy

nlp=spacy.load('de_core_news_sm')

# Text übergeben
text = (satoshis_paper)

# natural language processor mit unserem Text starten 
doc = nlp(text)

# NPs ausgeben, wir können auch direkt chunken
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

# 2. Möglichkeit, for-loop durch noun_chunks und text mit label ausgeben
for noun in doc.noun_chunks:
    print(noun.text, noun.label_)    

Noun phrases: ['Der Internethandel', 'Finanzinstituten', 'die', 'vertrauenswürdige Dritte', 'elektronische Zahlungen', 'dieses System', 'die meisten Transaktionen', 'den\ninhärenten Schwächen', 'des vertrauensbasierten Modells', 'Nicht rückgängig zu machende\nTransaktionen', 'Finanzinstitute', 'Streitigkeiten', 'Die Kosten', 'der Konfliktlösung', 'die Transaktionskosten', 'was', 'die kleinste praktikable Transaktionsgröße', 'damit die Möglichkeit', 'kleiner, \ngelegentlicher Transaktionen', 'die allgemeineren Kosten', 'des Verlustes', 'der Fähigkeit', 'irreversible Zahlungen', 'irreversible Dienstleistungen', 'der Möglichkeit', 'einer Rückgängigmachung', 'sich', 'die Notwendigkeit', 'des Vertrauens', 'Händler', 'ihren Kunden', 'ihnen', 'mehr Informationen', 'sie', 'Ein gewisser Anteil', 'Betrügerei', 'Diese Kosten', 'Zahlungsunsicherheiten', 'persönlichen Interaktionen', 'die Verwendung', 'Bargeld', 'kein Mechanismus', 'der', 'vertrauenswürdige \nDritte', 'Kommunikationskanäle', 'Was',

### SpaCy vs. NLTK: Precision & Recall

In [9]:
spaCy_list = []
for noun in doc.noun_chunks:
    spaCy_list.append(noun.text)

# Precision berechnen
precision = 0
for np in spaCy_list:
    if np in result_list:
        precision += 1
precision /= len(spaCy_list)

# Recall berechnen
recall = 0
for np in result_list:
    if np in spaCy_list:
        recall += 1
recall /= len(result_list)

# Print precision and recall
print("Precision:", precision)
print("Recall:", recall)

Precision: 0.527027027027027
Recall: 0.6964285714285714


### Fazit
Mein NLTK Modell hat eine hohe Übereinstimmung im Precision/Recall erzielt, wobei gesagt werden muss, dass dieses Modell bewusst und voreingenommen auf einen bereits gelesenen Text angewendet wurde. Auch die Nominalphrasen waren mir durch das händische extrahieren ja bereits bekannt. Das Modell wurde also speziell auf diese eine Einleitung angepasst aufgebaut. Das selbe Modell mit den selben regulären Ausdrücken könnte bei einem anderen deutschen Text schlechter abschneiden. Genau so könnte SpaCy hier viellecht sogar präziser sein. Bei eventuellem Einsatz von ML/LLM müssten diese Punkte berücksichtigt werden.